In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

# 1. MNIST 데이터셋 로드
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# 2. 데이터 정규화 (픽셀 밝기 값을 0~1 사이로)
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# -------------------------------------------------------------------
def add_coordinates(images):
  """이미지에 (x, y) 좌표 채널을 추가하는 함수"""
  num_images, height, width = images.shape

  # x, y 좌표 격자 생성 (0~27 값)
  x_coords = np.arange(width).astype('float32')
  y_coords = np.arange(height).astype('float32')
  xx, yy = np.meshgrid(x_coords, y_coords)

  # 좌표 값도 0~1 사이로 정규화
  xx = xx / (width - 1)
  yy = yy / (height - 1)

  # (밝기, y좌표, x좌표) 3개의 채널로 결합
  # images      shape: (num_images, 28, 28)
  # yy, xx      shape: (28, 28)
  # np.newaxis를 통해 yy와 xx를 (1, 28, 28)로 만든 뒤
  # 브로드캐스팅을 활용하여 모든 이미지에 동일한 좌표계를 더해줌
  images = images[:, :, :, np.newaxis] # (num_images, 28, 28, 1)
  coords = np.stack([yy, xx], axis=-1) # (28, 28, 2)
  coords = coords[np.newaxis, :, :, :] # (1, 28, 28, 2)
  coords = np.tile(coords, (num_images, 1, 1, 1)) # (num_images, 28, 28, 2)

  images_with_coords = np.concatenate([images, coords], axis=-1) # (num_images, 28, 28, 3)

  return images_with_coords

# 훈련 데이터와 테스트 데이터에 좌표 정보 추가
x_train_coords = add_coordinates(x_train)
x_test_coords = add_coordinates(x_test)

# Dense 층에 넣기 위해 1차원으로 펼치기
# (60000, 28, 28, 3)  -> (60000, 2352)
x_train_coords_flat = x_train_coords.reshape(x_train_coords.shape[0], -1)
x_test_coords_flat = x_test_coords.reshape(x_test_coords.shape[0], -1)

# -------------------------------------------------------------------

# 3. 모델 A: 기본 Dense 모델 (입력: 784개)
print("--- 모델 A (기본) 훈련 시작 ---")
model_A = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)), # 입력: (28, 28) -> 784
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10, activation='softmax')
])

model_A.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

model_A.fit(x_train, y_train, epochs=5, validation_split=0.2)
loss_A, acc_A = model_A.evaluate(x_test, y_test, verbose=2)


# 4. 모델 B: 네 아이디어 적용 모델 (입력: 2352개)
print("\n--- 모델 B (좌표 추가) 훈련 시작 ---")
model_B = tf.keras.models.Sequential([
    # Flatten 대신 InputLayer로 직접 입력 shape 지정
    tf.keras.layers.InputLayer(input_shape=(28 * 28 * 3,)), # 입력: 2352
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10, activation='softmax')
])

model_B.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

model_B.fit(x_train_coords_flat, y_train, epochs=5, validation_split=0.2)
loss_B, acc_B = model_B.evaluate(x_test_coords_flat, y_test, verbose=2)


# 5. 최종 결과 비교
print("\n" + "="*30)
print("          최종 결과 비교")
print("="*30)
print(f"모델 A (기본)      - 정확도: {acc_A*100:.2f}%")
print(f"모델 B (좌표 추가) - 정확도: {acc_B*100:.2f}%")
print("="*30)

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
--- 모델 A (기본) 훈련 시작 ---


/usr/local/lib/python3.12/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.8472 - loss: 0.5238 - val_accuracy: 0.9527 - val_loss: 0.1611
Epoch 2/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.9490 - loss: 0.1738 - val_accuracy: 0.9644 - val_loss: 0.1201
Epoch 3/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 11s 5ms/step - accuracy: 0.9630 - loss: 0.1210 - val_accuracy: 0.9702 - val_loss: 0.0949
Epoch 4/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.9705 - loss: 0.0995 - val_accuracy: 0.9732 - val_loss: 0.0902
Epoch 5/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9755 - loss: 0.0800 - val_accuracy: 0.9737 - val_loss: 0.0872
313/313 - 1s - 3ms/step - accuracy: 0.9743 - loss: 0.0834

--- 모델 B (좌표 추가) 훈련 시작 ---


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Epoch 1/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 9s 6ms/step - accuracy: 0.7128 - loss: 0.9128 - val_accuracy: 0.9111 - val_loss: 0.3002
Epoch 2/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 9s 6ms/step - accuracy: 0.8784 - loss: 0.3943 - val_accuracy: 0.9226 - val_loss: 0.2539
Epoch 3/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 10s 6ms/step - accuracy: 0.8980 - loss: 0.3395 - val_accuracy: 0.9338 - val_loss: 0.2262
Epoch 4/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 10s 6ms/step - accuracy: 0.9055 - loss: 0.3140 - val_accuracy: 0.9449 - val_loss: 0.1943
Epoch 5/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 8s 6ms/step - accuracy: 0.9104 - loss: 0.2929 - val_accuracy: 0.9463 - val_loss: 0.1830
313/313 - 1s - 4ms/step - accuracy: 0.9436 - loss: 0.1780

          최종 결과 비교
모델 A (기본)      - 정확도: 97.43%
모델 B (좌표 추가) - 정확도: 94.36%


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
import time

# 1) MNIST 데이터셋 로드
transform = transforms.Compose([
    transforms.ToTensor()
])
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset  = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

# ---------------------------------------------------------------------------------
# 2) 모델 A (기본)를 위한 데이터 로더
# ---------------------------------------------------------------------------------
train_loader_A = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader_A  = torch.utils.data.DataLoader(test_dataset, batch_size=1000, shuffle=False)

# ---------------------------------------------------------------------------------
# 3) 모델 B (좌표 추가)를 위한 데이터 전처리와 로더
# ---------------------------------------------------------------------------------
def add_coordinates(img_tensor):
    img_tensor = img_tensor.squeeze(0)
    h, w = img_tensor.shape
    y_coords, x_coords = torch.meshgrid(torch.arange(h), torch.arange(w), indexing="ij")
    x_coords = x_coords.float() / (w - 1)
    y_coords = y_coords.float() / (h - 1)
    out = torch.stack([img_tensor, y_coords, x_coords], dim=2)  # 채널 순서 (밝기, y, x)
    return out.view(-1)

class CoordMNIST(torch.utils.data.Dataset):
    def __init__(self, dataset):
        self.dataset = dataset
    def __len__(self):
        return len(self.dataset)
    def __getitem__(self, idx):
        img, label = self.dataset[idx]
        img_with_coords = add_coordinates(img)
        return img_with_coords, label

train_loader_B = torch.utils.data.DataLoader(CoordMNIST(train_dataset), batch_size=64, shuffle=True)
test_loader_B  = torch.utils.data.DataLoader(CoordMNIST(test_dataset), batch_size=1000, shuffle=False)

# ---------------------------------------------------------------------------------
# 4) 모델 정의
# ---------------------------------------------------------------------------------
device   = torch.device("cuda" if torch.cuda.is_available() else "cpu")
criterion = nn.CrossEntropyLoss()

# 모델 A: 기본 Dense 모델 (입력: 784)
class DenseNet_A(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.fc1     = nn.Linear(784, 128)
        self.dropout = nn.Dropout(0.2)
        self.fc2     = nn.Linear(128, 10)
    def forward(self, x):
        x = self.flatten(x)
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

# 모델 B: 좌표 추가 Dense 모델 (입력: 784*3)
class DenseNet_B(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1     = nn.Linear(784*3, 128)  # 입력 크기만 다름
        self.dropout = nn.Dropout(0.2)
        self.fc2     = nn.Linear(128, 10)
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

# ---------------------------------------------------------------------------------
# 5) 훈련 및 평가 함수 정의 (코드를 재사용하기 위해)
# ---------------------------------------------------------------------------------
def train_and_evaluate(model, train_loader, test_loader, epochs=3):
    model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=1e-3)

    # 훈련 루프
    for epoch in range(epochs):
        model.train()
        for imgs, labels in train_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            optimizer.zero_grad()
            output = model(imgs)
            loss = criterion(output, labels)
            loss.backward()
            optimizer.step()
        print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")

    # 평가 루프
    model.eval()
    correct = 0
    total   = 0
    with torch.no_grad():
        for imgs, labels in test_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            output = model(imgs)
            preds  = output.argmax(dim=1)
            correct += (preds == labels).sum().item()
            total   += labels.size(0)

    accuracy = 100 * correct / total
    return accuracy

# ---------------------------------------------------------------------------------
# 6) 각 모델 실행 및 결과 저장
# ---------------------------------------------------------------------------------
print("--- [PyTorch] 모델 A (기본) 훈련 시작 ---")
model_A = DenseNet_A()
acc_A   = train_and_evaluate(model_A, train_loader_A, test_loader_A)

print("\n--- [PyTorch] 모델 B (좌표 추가) 훈련 시작 ---")
model_B = DenseNet_B()
acc_B   = train_and_evaluate(model_B, train_loader_B, test_loader_B)

# ---------------------------------------------------------------------------------
# 7) 최종 결과 비교
# ---------------------------------------------------------------------------------
print("\n" + "="*40)
print("          PyTorch 최종 결과 비교")
print("="*40)
print(f"모델 A (기본)      - 정확도: {acc_A:.2f}%")
print(f"모델 B (좌표 추가) - 정확도: {acc_B:.2f}%")
print("="*40)

100%|██████████| 9.91M/9.91M [00:00<00:00, 54.2MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 1.59MB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 13.9MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 6.70MB/s]


--- [PyTorch] 모델 A (기본) 훈련 시작 ---
Epoch 1, Loss: 0.0879
Epoch 2, Loss: 0.0554
Epoch 3, Loss: 0.0313

--- [PyTorch] 모델 B (좌표 추가) 훈련 시작 ---
Epoch 1, Loss: 0.2947
Epoch 2, Loss: 0.4104
Epoch 3, Loss: 0.4457

          PyTorch 최종 결과 비교
모델 A (기본)      - 정확도: 97.00%
모델 B (좌표 추가) - 정확도: 93.12%
